In [3]:
source("Distance_Functions.r")
library(reshape2)

In [34]:
dist_funcs = c(mseMatrix, euclidMatrix, adistMatrix, manMatrix, canMatrix)
dist_funcs_name = c("mse", "euclid", "adist", "manhattan", "canberra")

for (chants in dir("./Chants")){
    if (!(chants == 'README.md')){
        
        curr_chant = read.csv(file=paste("./Chants/",chants, sep="" ), header=TRUE, sep=',')
        dist = 1
    
        for (func in dist_funcs){
            assign(paste(dist_funcs_name[dist],gsub(" ","",strsplit(chants,".csv")),sep=""), func(curr_chant))
            dist = dist + 1
        }
        dist = 1
    }
}
 
while (dist < 6){
    assign(paste(dist_funcs_name[dist], "fullmat", sep = ""),
                acast(rbind(melt(eval(as.symbol(paste(dist_funcs_name[dist],"BenediciteDominum",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"DominusDixit",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"FactusEst",sep="")))),
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"LexDomini",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"NomineDomini",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"Rorate", sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"SicutOculi", sep=""))))), Var1~Var2, sum))
    
    temp = eval(as.symbol(paste(dist_funcs_name[dist], "fullmat", sep="")))
    temp[temp==0] <- NA
    
    diag(temp) <- 0
    
    write.table(temp, file=paste(paste(dist_funcs_name[dist], "_distance/", sep=""), 
                            paste(dist_funcs_name[dist], "fullmat", sep = ""), sep=""))
    dist = dist + 1  
}

In [50]:
read.table("mse_distance/msefullmat")

,Aqu,Dij,Arne,Nev,NIt,Gai,MCa,Rome3,Ger1,Sens,Rome2,VimA,Ger2,VimC,Che,Rou,Aux
Aqu,0.0000000,3.8706197,4.0383606,4.9174938,2.2165476,5.3587236,5.687139,36.319105,3.2836957,1.9829427,3.145631,6.900858,0.6226415,1.660377,1.5463515,0.9917355,0.6718750
Dij,3.8706197,0.0000000,2.3801099,3.5128230,1.8156746,3.6637210,4.067663,33.775310,2.3817229,1.0546770,NA,5.133953,0.6289308,1.389937,1.5020181,0.6942149,0.5833333
Arne,4.0383606,2.3801099,0.0000000,3.4392783,1.8255060,3.8803054,4.257776,33.135308,2.5117119,1.2960086,2.407767,4.603236,0.6540881,1.320755,0.5950413,0.8842975,0.5625000
Nev,4.9174938,3.5128230,3.4392783,0.0000000,1.6923313,4.9927685,5.192523,36.254036,3.5164479,1.8216412,2.320388,6.353400,0.6477987,1.301887,1.5582677,0.7190083,0.3697917
NIt,2.2165476,1.8156746,1.8255060,1.6923313,0.0000000,2.2692063,1.963016,25.095456,0.7343750,0.5911111,NA,3.155556,NA,NA,NA,NA,0.6145833
Gai,5.3587236,3.6637210,3.8803054,4.9927685,2.2692063,0.0000000,5.077458,35.582597,3.1731046,1.9222970,2.281553,6.854117,0.7672956,1.377358,1.2663848,0.8264463,0.7291667
MCa,5.6871389,4.0676633,4.2577756,5.1925229,1.9630159,5.0774584,0.000000,34.161859,3.8601173,2.2321608,2.349515,6.740431,0.7547170,1.402516,2.1203152,1.0495868,0.5000000
Rome3,36.3191049,33.7753097,33.1353075,36.2540355,25.0954563,35.5825975,34.161859,0.000000,12.6754845,25.0342641,1.155340,22.973150,1.7106918,2.496855,7.5052213,3.8181818,2.1250000
Ger1,3.2836957,2.3817229,2.5117119,3.5164479,0.7343750,3.1731046,3.860117,12.675485,0.0000000,1.5676225,2.330097,3.787838,0.5220126,1.522013,1.3981037,1.0413223,0.6406250
Sens,1.9829427,1.0546770,1.2960086,1.8216412,0.5911111,1.9222970,2.232161,25.034264,1.5676225,0.0000000,2.417476,6.582552,NA,NA,0.7906977,NA,NA


In [59]:
dist = 1
assign(paste(dist_funcs_name[dist], "fullmat", sep = ""),
                acast(rbind(melt(eval(as.symbol(paste(dist_funcs_name[dist],"BenediciteDominum",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"DominusDixit",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"FactusEst",sep="")))),
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"LexDomini",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"NomineDomini",sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"Rorate", sep="")))), 
                            melt(eval(as.symbol(paste(dist_funcs_name[dist],"SicutOculi", sep=""))))), Var1~Var2, median))

In [60]:
eval(as.symbol(paste(dist_funcs_name[dist], "fullmat", sep = "")))

,Aqu,Dij,Arne,Nev,NIt,Gai,MCa,Rome3,Ger1,Sens,Rome2,VimA,Ger2,VimC,Che,Rou,Aux
Aqu,0.0000000,0.6888889,0.7106918,0.7106918,0.7916667,0.8113208,0.7555556,2.930233,0.6289308,0.6511628,3.145631,2.302326,0.6226415,1.660377,0.7731757,0.9917355,0.6718750
Dij,0.6888889,0.0000000,0.4533333,0.6037317,0.6222222,0.5564438,0.6267996,3.362576,0.6453854,0.5273385,NA,2.566977,0.6289308,1.389937,0.7510090,0.6942149,0.5833333
Arne,0.7106918,0.4533333,0.0000000,0.5978197,0.5982143,0.6719733,0.6771411,3.002384,0.6214508,0.6480043,2.407767,2.301618,0.6540881,1.320755,0.5950413,0.8842975,0.5625000
Nev,0.7106918,0.6037317,0.5978197,0.0000000,0.4947917,0.6899225,0.7232143,2.945736,0.6821705,0.5955556,2.320388,1.976744,0.6477987,1.301887,0.7791338,0.7190083,0.3697917
NIt,0.7916667,0.6222222,0.5982143,0.4947917,0.0000000,0.7500000,0.6177778,3.883929,0.7343750,0.5911111,NA,3.155556,NA,NA,NA,NA,0.6145833
Gai,0.8113208,0.5564438,0.6719733,0.6899225,0.7500000,0.0000000,0.5892857,2.899225,0.5785124,0.5503876,2.281553,2.085271,0.7672956,1.377358,0.6331924,0.8264463,0.7291667
MCa,0.7555556,0.6267996,0.6771411,0.7232143,0.6177778,0.5892857,0.0000000,2.868217,0.7547170,0.8349515,2.349515,2.255814,0.7547170,1.402516,1.0601576,1.0495868,0.5000000
Rome3,2.9302326,3.3625761,3.0023838,2.9457364,3.8839286,2.8992248,2.8682171,0.000000,2.1262136,3.1317829,1.155340,3.294574,1.7106918,2.496855,3.7526107,3.8181818,2.1250000
Ger1,0.6289308,0.6453854,0.6214508,0.6821705,0.7343750,0.5785124,0.7547170,2.126214,0.0000000,0.7838112,2.330097,1.893919,0.5220126,1.522013,0.6990518,1.0413223,0.6406250
Sens,0.6511628,0.5273385,0.6480043,0.5955556,0.5911111,0.5503876,0.8349515,3.131783,0.7838112,0.0000000,2.417476,2.007752,NA,NA,0.7906977,NA,NA


[1] 53.38

In [56]:
b = c(1,2,3,4,4,5,5,5,6,4,3,2)

In [57]:
mean(b)

[1] 3.666667

In [58]:
median(b)

[1] 4